Only run if in Colab!

In [ ]:
!git clone https://github.com/ratinac-nikola/rbm.git
%cd rbm
%pip install -r requirements.txt

In [ ]:
%pwd

### Initialization

In [1]:
from sklearn.model_selection import train_test_split
import torch
import config

if torch.cuda.is_available():
    print("Using GPU")
    device = torch.device('cuda')
else:
    print("Using CPU")
    device = "cpu"

Using CPU


In [2]:
from dataset import Dataset
dataset = Dataset(device=device)
tensorData = dataset.getDatasetAsTensor()
trainset, testset = train_test_split(tensorData, train_size=0.8)
trainset.shape, testset.shape

(torch.Size([754, 1682, 5]), torch.Size([189, 1682, 5]))

In [3]:
from rbm import RBM
rbm = RBM(tensorData.shape[1], n_hidden=10, device=device)

In [12]:
def strip(v):
    return v[v.sum(dim=1) > 0]

def RMSE(v0, vk):
    return torch.sqrt(mse(v0,vk)**2).item()

In [13]:
mse = torch.nn.MSELoss()
recon_err = []
for case in testset:
        rec = rbm.reconstruct(case)
        rec[case.sum(dim=1) == 0] = case[case.sum(dim=1) == 0]
        rec = strip(rec)
        case = strip(case)
        case = torch.argmax(case, dim=1).float()
        rec = torch.argmax(rec, dim=1).float()
        
        recon_err += [RMSE(case, rec)]
print(sum(recon_err)/len(recon_err))
for epoch in range(1, config.epochs + 1):
    i = 1
    recon_err = []
    c = 1
    for case in trainset:
        
        v0 = case
        vk = case
        c += 1
        for k in range(2):
            vk = rbm.reconstruct(vk)

            vk[v0.sum(dim=1) == 0] = v0[v0.sum(dim=1) == 0]
        
        rbm.train(v0, vk)
        
        vv = rbm.reconstruct(v0)
        vv[v0.sum(dim=1) == 0] = v0[v0.sum(dim=1) == 0]
        vv = strip(vv)
        v0 = strip(v0)
        v0 = torch.argmax(v0, dim=1).float()
        vv = torch.argmax(vv, dim=1).float()
        
        recon_err += [RMSE(v0, vv)]
        
    print(f"Epoch {epoch} done. Mean reconstruction RMSE: {sum(recon_err)/len(recon_err)}")
    
    recon_err = []
    for case in testset:
        rec = rbm.reconstruct(case)
        rec[case.sum(dim=1) == 0] = case[case.sum(dim=1) == 0]
        rec = strip()
        case = strip()
        case = torch.argmax(case, dim=1).float()
        rec = torch.argmax(rec, dim=1).float()
        
        recon_err += [RMSE(v0, vv)]
    
    print(f"Testset reconstruction: {sum(recon_err)/len(recon_err)}")
        
        
    


3.2893042444552063


KeyboardInterrupt: 